In [ ]:
# Keras
from tensorflow import keras
from keras import layers
from tensorflow.keras.models import Model

import cuda

#SciKit
from sklearn.metrics import confusion_matrix

#matplot
import matplotlib.pyplot as plt

#Numpy
import numpy as np

# Loading [Mnist](https://keras.io/api/datasets/mnist/) dataset

In [ ]:

np.random.seed(123)
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

x_train = x_train/255
x_test =x_test/255

y_train_onehot = keras.utils.to_categorical(y_train, 10)
y_test_onehot = keras.utils.to_categorical(y_test, 10)

In [ ]:
img = x_train[0]
plt.imshow(img, cmap='gray')
plt.show()

# LeNet

In [ ]:
model = keras.Sequential()

model.add(layers.Conv2D(filters=6, kernel_size=(5, 5), activation='relu', input_shape=(28,28,1)))
model.add(layers.AveragePooling2D())

model.add(layers.Conv2D(filters=16, kernel_size=(5, 5), activation='relu'))
model.add(layers.AveragePooling2D())

model.add(layers.Flatten())

model.add(layers.Dense(units=120, activation='relu'))

model.add(layers.Dense(units=84, activation='relu'))

model.add(layers.Dense(units=10, activation = 'softmax'))

In [ ]:
model.build()
model.summary()

## Training Phase

In [ ]:
precision = keras.metrics.Precision()
recall = keras.metrics.Recall()
model.compile(loss='categorical_crossentropy',
            optimizer='adam',
            metrics=['accuracy', precision, recall])

history = model.fit(x_train, y_train_onehot,
          batch_size=128, epochs=5, verbose=1)

## Test Phase

Evaluating metrics between testing and training phases.

In [ ]:
score = model.evaluate(x_test, y_test_onehot, verbose=0)

In [ ]:
for metric in ['accuracy', 'precision_2', 'recall_2']:
    plt.plot(history.epoch, history.history[metric], label=metric+'_train')


plt.plot(history.epoch, np.full(5, score[1]), label='accuracy_test')
plt.plot(history.epoch, np.full(5, score[2]), label='precision_test')
plt.plot(history.epoch, np.full(5, score[3]), label='recall_test')

plt.legend()
plt.xlabel('train epoch')
plt.show()

Confusion Matrix

In [ ]:
ŷ = model.predict(x_test)
y_pred = [np.argmax(yi) for yi in ŷ]
confusion_matrix(y_test, y_pred)

## Visualizing Feature maps

In [ ]:
def visualizeFeatureMap(img, model, nth_layer):
    # visualize the feature map off input<img> at model<model> on convolution layer<nth_layer>

    # Create a model that will return these outputs, given the model input
    feature_map_model = Model(inputs=model.inputs, outputs=model.layers[nth_layer].output)

    # Use the model to predict the features
    feature_maps = feature_map_model.predict(
        img.reshape(1, img.shape[0], img.shape[1], img.shape[2] if len(img.shape)==3 else 1), verbose = 0)


    numOfFeatureMaps = model.layers[nth_layer].output.shape[3]
    gridAxisDim = np.int16(np.ceil(np.sqrt(numOfFeatureMaps)))
    for i in range(numOfFeatureMaps):
        plt.subplot(gridAxisDim, gridAxisDim, i+1)
        plt.imshow(feature_maps[0, :, :, i], cmap='viridis')
    plt.show()

In [ ]:
visualizeFeatureMap(img, model, 0)

In [ ]:
for i in range(10):
    num = i
    filter = y_test == num

    x_test_num = x_test[filter]
    y_test_num = y_test[filter]

    visualizeFeatureMap(x_test_num[0], model, 0)
    visualizeFeatureMap(x_test_num[0], model, 2)
    print('----------------------------------------------------------')
    visualizeFeatureMap(x_test_num[1], model, 0)
    visualizeFeatureMap(x_test_num[1], model, 2)
    print('----------------------------------------------------------')


# Conclusion

The CNN LeNet aproach for MNIST classification problem proved to be better than MLP because the number of parameters.

LeNet has 44.426 parameters and my MLP approach has 1.024.080.

